In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  


In [2]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

### Extract date



In [3]:
url = 'https://www.moh.gov.sg/covid-19'

driver.get(url)
content = driver.page_source


In [4]:
soup =BeautifulSoup(content, 'lxml')

In [5]:
data = soup.findAll(text=re.compile("Case Summary in Singapore \(as of .+"))


In [6]:
data

['Case Summary in Singapore (as of 2 Mar 2020, 1200h)']

In [7]:
m = re.search("(\d+ \w+ 202\d, \d+\d+h)", data[0])
date_time_str = m.group(1)
data_date_time = datetime.strptime(date_time_str, '%d %b %Y, %H%Mh').strftime('%Y-%m-%d')
data_date_time


'2020-03-02'

### Extract Dataframes

In [8]:
dfs = pd.read_html(content)

In [9]:
dfs

[    0   1
 0 NaN NaN
 1 NaN NaN,
     0   1
 0 NaN NaN
 1 NaN NaN,
                        0                        1
 0           ACTIVE CASES             ACTIVE CASES
 1                     32                       32
 2  Hospitalised (Stable)  Hospitalised (Critical)
 3                     25                        7
 4                  Death               Discharged
 5                      0                       78,
                0       1
 0  DORSCON Level  Orange,
               0                                                  1
 0          Date                                              Title
 1    3 Mar 2020      Two New Cases of COVID-19 Infection Confirmed
 2    3 Mar 2020  Remarks by Minister Lawrence Wong, Co-chair of...
 3    3 Mar 2020  Additional Precautionary Measures in Response ...
 4    2 Mar 2020  Four More Cases Discharged; Two New Cases of C...
 5    1 Mar 2020  Two More Cases Discharged; Four New Cases of C...
 6   29 Feb 2020  Three More Cases Discharged

In [10]:
# Look for df with confirmed cases, tested negative, ...

for df in dfs:
    combined_fields = df[1].to_string().lower()
    if "active cases" in combined_fields and "discharged" in combined_fields:
        sing_cases_df_temp = df
        
sing_cases_df_temp

,0,1
0,ACTIVE CASES,ACTIVE CASES
1,32,32
2,Hospitalised (Stable),Hospitalised (Critical)
3,25,7
4,Death,Discharged
5,0,78


In [11]:
# very hacky to workaround with the change from MoH site
sing_cases_df = pd.DataFrame({'active_cases': [int(sing_cases_df_temp[0][1])],
                             'hospitalised_stable': [int(sing_cases_df_temp[0][3])],
                              'hospitalised_critical': [int(sing_cases_df_temp[1][3])],
                              'death': [int(sing_cases_df_temp[0][5])],
                              'discharge': [int(sing_cases_df_temp[1][5])],
                              'datetime': data_date_time
                             },
                            )
sing_cases_df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,32,25,7,0,78,2020-03-02


In [12]:
df = sing_cases_df
df

,active_cases,hospitalised_stable,hospitalised_critical,death,discharge,datetime
0,32,25,7,0,78,2020-03-02


### Save to file

In [13]:
from pathlib import Path
csv_file = 'data/singapore-cases.csv'

if Path(csv_file).exists():
    # read out the old data
    old_df = pd.read_csv(csv_file)
    df = df.set_index('datetime').append(old_df.set_index('datetime'))



/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [14]:
df = df.reset_index()
df.drop_duplicates(subset=['datetime'], inplace=True)

In [15]:
df.sort_values(by=['datetime'], inplace=True)
df

,datetime,active_cases,confirmed,death,discharge,discharged,hospitalised,hospitalised_critical,hospitalised_stable,negative,pending
1,2020-02-02,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,240.0,43.0
2,2020-02-03,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,262.0,32.0
3,2020-02-04,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,289.0,20.0
4,2020-02-05,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,295.0,62.0
5,2020-02-06,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,310.0,147.0
6,2020-02-07,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,363.0,181.0
7,2020-02-08,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,438.0,181.0
8,2020-02-09,NaN,45.0,NaN,NaN,7.0,NaN,NaN,NaN,581.0,39.0
9,2020-02-10,NaN,45.0,NaN,NaN,7.0,NaN,NaN,NaN,581.0,39.0
10,2020-02-11,NaN,47.0,NaN,NaN,9.0,NaN,NaN,NaN,608.0,43.0


In [16]:
df.to_csv(csv_file, index=False)
